In [1]:
import warnings
warnings.filterwarnings("ignore")

import json
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
plt.rc('pdf', fonttype=42) # Avoid type 3 fonts

sns.set_theme(style="white")
sns.set_palette(sns.color_palette("tab10"))
plt.rcParams.update({"xtick.bottom" : True}) # enables ticks

In [5]:
#
# Android data
#

ANDROID_BASE_DIR = '../android'
ANDROID_RESULTS_DIR = os.path.join(ANDROID_BASE_DIR, '../results/small-hw-support-2024-05-24-161053/')

ANDROID_TESTS = {
    "small": "testBenchLongSloth_withOwasp_small",
    "large": "testBenchLongSloth_withOwasp_large",
}


def android_read_device_info(path):
    with open(path) as f:
        x = json.load(f)
        
    name = x['name'].replace('"', '')
    name = name.replace(' 5G', '')
        
    os = x['os']
    if '.' in os:
        os = os[:os.find('.')]
    
    return {
        'name': name,
        'os': int(os),
    }


def android_read_sloth_bench_results(dir_path, test_name):
    allL, allDurations = [], []
    
    with open(os.path.join(dir_path, f'{test_name}.json')) as f:
        x = json.load(f)
        
        for m in x:
            d = m['data']
            if m['tag'] != 'LogTracer' or test_name not in d:
                continue
            _, _, l, _, _, afterPwHashNs, _, finishNs = d.split(' ')
    
            allL.append(int(l.split('=')[1]))
        
            afterPwHashNs = int(afterPwHashNs.split('=')[1])
            finishNs = int(finishNs.split('=')[1])
            allDurations.append((finishNs - afterPwHashNs) / 1_000_000) # NS to MS
    
    return {
        'l': allL,
        'duration': allDurations, 
    }


def android_read_job_result(job_name, job_path, test_id, test_name):
    device_info = android_read_device_info(os.path.join(job_path, "device.json"))
    bench_results = android_read_sloth_bench_results(job_path, test_name)
    
    return {**device_info, **{'test': test_id}, **bench_results}

                  
def android_read_all_jobs():
    all_data = []
    for job_name in sorted(os.listdir(ANDROID_RESULTS_DIR)):
        job_path = os.path.join(ANDROID_RESULTS_DIR, job_name)
        
        if not os.path.isdir(job_path):
            continue
            
        for test_id, test_name in ANDROID_TESTS.items():
            x = android_read_job_result(job_name, job_path, test_id, test_name)
            if len(x['l']) < 10: continue
            all_data.append(x)
    return pd.json_normalize(all_data)
        
android_df = android_read_all_jobs()
print(android_df)
android_df = android_df.explode(['l', 'duration'], ignore_index=True)
android_df = android_df.astype({'duration': 'float64'})

android_df.sample(20)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



KeyError: 'l'

In [ ]:
#
# iOS data
#

IOS_BASE_DIR = 'ios'
IOS_DATA_DIR = os.path.join(IOS_BASE_DIR, 'server/data')

IOS_FILES = [
    'CHANGE-ME.json',
    'CHANGE-ME.json',
]


def ios_get_device_info(device_name):
    with open(os.path.join(IOS_BASE_DIR, 'models.json')) as f:
        return json.load(f)[device_name]


def ios_read_data(path):
    with open(path) as f:
        d = json.load(f)
        
    info = ios_get_device_info(d['device'])
    
    return pd.DataFrame(data={
        'Device': info['name'],
        'OS Version': d['version'],
        'Chip': info['chip'],
        'Configuration': f"{info['name']} ({d['version']})",
        'n': d['n'],
        'type': 'small' if d['n'] < 25 else 'large',
        'Measurement': [float(x) * 1000 for x in d['executionTimeSeconds']],
    })


def ios_read_all_data():
    for f in IOS_FILES:
        path = os.path.join(IOS_DATA_DIR, f)
        x = ios_read_data(path)
        yield x
        
ios_df = pd.concat(ios_read_all_data())
ios_df.sample(10)

In [ ]:
#
# Box plot with the Android LongSloth execution times
#

fig, axs = plt.subplots(2,2)
fig.set_size_inches((5.5, 3.3))

((ax1, ax2), (ax3, ax4)) = axs

#
# Android
#

sns.boxplot(
    data=android_df[android_df.test=='small'],
    y='name',
    x='duration',
    hue='name',
    ax=ax1,
    showfliers = False,
    dodge=False,
)
ax1.axvline(x=67, color='r')
ax1.set_xlabel(None)
ax1.set_ylabel(None)
ax1.grid(ls='--')
ax1.legend().remove()


sns.boxplot(
    data=android_df[android_df.test=='large'],
    y='name',
    x='duration',
    hue='name',
    ax=ax2,
    showfliers = False,
    dodge=False,
)
ax2.axvline(x=555, color='r')
ax2.set_xlabel(None)
ax2.set_yticklabels([])
ax2.set_ylabel(None)
ax2.grid(ls='--')
ax2.legend().remove()

#
# iOS
#

sns.boxplot(
    data=ios_df[ios_df.type=='small'],
    y='Device',
    x='Measurement',
    color='white',
    ax=ax3,
    showfliers = False,
    boxprops = {'edgecolor': 'black'},
    medianprops = {'color': 'black'},
    whiskerprops = {'color': 'black'},
    capprops = {'color': 'black'},
)
ax3.axvline(x=67, color='r')
ax3.set_xlabel("Duration [ms]")
ax3.set_ylabel(None)
ax3.grid(ls='--')


sns.boxplot(
    data=ios_df[ios_df.type=='large'],
    y='Device',
    x='Measurement',
    color='white',
    ax=ax4,
    showfliers = False,
    boxprops = {'edgecolor': 'black'},
    medianprops = {'color': 'black'},
    whiskerprops = {'color': 'black'},
    capprops = {'color': 'black'},
)
ax4.axvline(x=555, color='r')
ax4.set_xlabel("Duration [ms]")
ax4.set_yticklabels([])
ax4.set_ylabel(None)
ax4.grid(ls='--')


#
# Common
#

ax1.set_xlim((40, 210))
ax3.set_xlim((40, 210))
ax2.set_xlim((480, 820))
ax4.set_xlim((480, 820))

fig.tight_layout()
fig.savefig("output/android-and-ios-sloth-perf.pdf", dpi=300, bbox_inches='tight')